In [1]:
import unittest
import os
import pickle
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely

import sandpyper
from sandpyper.sandpyper import ProfileSet, ProfileDynamics
from sandpyper.common import get_sil_location, get_opt_k, create_transects, sensitivity_tr_rbcd

pd.options.mode.chained_assignment = None  # default='warn'

C:\anaconda3\envs\sandpyper_env\lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
C:\anaconda3\envs\sandpyper_env\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
# create global variables used across the test analysis
loc_codes=["mar","leo"]

loc_full={'mar': 'Marengo',
         'leo': 'St. Leonards'}

loc_specs={'mar':{'thresh':6,
       'min_points':6}}

loc_search_dict = {'leo': ['St', 'Leonards', 'leonards', 'leo'], 'mar': ['Marengo', 'marengo', 'mar'] }
crs_dict_string = {'mar': {'init': 'epsg:32754'}, 'leo':{'init': 'epsg:32755'} }

labels=["Undefined", "Small", "Medium", "High", "Extreme"]

appendix=["_deposition", "_erosion"]

relabel_dict={"Undefined_erosion":"ue",
            "Small_erosion":"se",
            "Medium_erosion":"me",
            "High_erosion":"he",
            "Extreme_erosion":"ee",
             "Undefined_deposition":"ud",
             "Small_deposition":"sd",
             "Medium_deposition":"md",
             "High_deposition":"hd",
             "Extreme_deposition":"ed"
            }

transects_spacing=20

feature_set=["band1","band2","band3","distance"]

water_dict={'leo_20180606':[0,9,10],
'leo_20180713':[0,3,4,7],
'leo_20180920':[0,2,6,7],
'leo_20190211':[0,2,5],
'leo_20190328':[2,4,5],
'leo_20190731':[0,2,8,6],
'mar_20180601':[1,6],
'mar_20180621':[4,6],
'mar_20180727':[0,5,9,10],
'mar_20180925':[6],
'mar_20181113':[1],
'mar_20181211':[4],
'mar_20190205':[],
'mar_20190313':[],
'mar_20190516':[4,7]}

no_sand_dict={'leo_20180606':[5],
'leo_20180713':[],
'leo_20180920':[],
'leo_20190211':[1],
'leo_20190328':[],
'leo_20190731':[1],
'mar_20180601':[4,5],
'mar_20180621':[3,5],
'mar_20180727':[4,7],
'mar_20180925':[5],
'mar_20181113':[0],
'mar_20181211':[0],
'mar_20190205':[0,5],
'mar_20190313':[4],
'mar_20190516':[2,5]}

veg_dict={'leo_20180606':[1,3,7,8],
'leo_20180713':[1,5,9],
'leo_20180920':[1,4,5],
'leo_20190211':[4],
'leo_20190328':[0,1,6],
'leo_20190731':[3,7],
'mar_20180601':[0,7],
'mar_20180621':[1,7],
'mar_20180727':[1,3],
'mar_20180925':[1,3],
'mar_20181113':[3],
'mar_20181211':[2],
'mar_20190205':[3],
'mar_20190313':[1,5],
'mar_20190516':[0]}

sand_dict={'leo_20180606':[2,4,6],
'leo_20180713':[2,6,8],
'leo_20180920':[3],
'leo_20190211':[3],
'leo_20190328':[3],
'leo_20190731':[4,5],
'mar_20180601':[2,3],
'mar_20180621':[0,2],
'mar_20180727':[2,6,8],
'mar_20180925':[0,4,2],
'mar_20181113':[2,4],
'mar_20181211':[3,1],
'mar_20190205':[1,2,4],
'mar_20190313':[0,2,3],
'mar_20190516':[1,3,6]}


l_dicts={'no_sand': no_sand_dict,
         'sand': sand_dict,
        'water': water_dict,
        'veg':veg_dict}

In [3]:
dsms_dir_path = os.path.abspath("examples/test_data/dsm_1m/")
orthos_dir_path = os.path.abspath("examples/test_data/orthos_1m")

transects_path = os.path.abspath("examples/test_data/transects")
lod_mode=os.path.abspath("examples/test_data/lod_transects/")

label_corrections_path=os.path.abspath("examples/test_data/clean/label_corrections.gpkg")
watermasks_path=os.path.abspath("examples/test_data/clean/watermasks.gpkg")
shoremasks_path=os.path.abspath("examples/test_data/clean/shoremasks.gpkg")

In [4]:
P = ProfileSet(dirNameDSM=dsms_dir_path,
                        dirNameOrtho=orthos_dir_path,
                        dirNameTrans=transects_path,
                        transects_spacing=transects_spacing,
                        loc_codes=loc_codes,
                        loc_search_dict=loc_search_dict,
                        crs_dict_string=crs_dict_string,
                        check="all")


C:\anaconda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


dsm from leo = 6

ortho from leo = 6

dsm from mar = 9

ortho from mar = 9


umber OF DATASETS TO PROCESS: 30


In [5]:
np.random.seed(42)
P.extract_profiles(mode='all',tr_ids='tr_id',sampling_step=1,add_xy=True,lod_mode=lod_mode)

Extracting elevation from DSMs . . .


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Extraction successful
Number of points extracted:1395
Time for processing=6.215153932571411 seconds
First 10 rows are printed below
Number of points outside the raster extents: 27
The extraction assigns NaN.
Number of points in NoData areas within the raster extents: 0
The extraction assigns NaN.


In [6]:
np.random.seed(10)
sil_df = get_sil_location(P.profiles,
                        ks=(2,15),
                        feature_set=feature_set,
                       random_state=10)

opt_k = get_opt_k(sil_df, sigma=0 )

P.kmeans_sa(opt_k, feature_set=feature_set)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Working on : mar, 20190516.


  0%|          | 0/13 [00:00<?, ?it/s]

For n_clusters = 2 The average silhouette_score is : 0.6219865168763407
For n_clusters = 3 The average silhouette_score is : 0.535782834177223
For n_clusters = 4 The average silhouette_score is : 0.5360693626824162
For n_clusters = 5 The average silhouette_score is : 0.4560535847617118
For n_clusters = 6 The average silhouette_score is : 0.45529082978244856
For n_clusters = 7 The average silhouette_score is : 0.44623104595368007
For n_clusters = 8 The average silhouette_score is : 0.4276467073078296
For n_clusters = 9 The average silhouette_score is : 0.39984472497160034
For n_clusters = 10 The average silhouette_score is : 0.3944833658664287
For n_clusters = 11 The average silhouette_score is : 0.39290037915600995
For n_clusters = 12 The average silhouette_score is : 0.3626866522843635
For n_clusters = 13 The average silhouette_score is : 0.3537214969510808
For n_clusters = 14 The average silhouette_score is : 0.360049558138258
Working on : mar, 20190313.


...

  0%|          | 0/13 [00:00<?, ?it/s]

For n_clusters = 2 The average silhouette_score is : 0.4012079287817844
For n_clusters = 3 The average silhouette_score is : 0.4563078554750914
For n_clusters = 4 The average silhouette_score is : 0.40640166442880776
For n_clusters = 5 The average silhouette_score is : 0.4001757072050638
For n_clusters = 6 The average silhouette_score is : 0.3890044660503746
For n_clusters = 7 The average silhouette_score is : 0.3716889344520391
For n_clusters = 8 The average silhouette_score is : 0.36600629265911533
For n_clusters = 9 The average silhouette_score is : 0.3503569988098764
For n_clusters = 10 The average silhouette_score is : 0.34651080656561367
For n_clusters = 11 The average silhouette_score is : 0.34322073624826754
For n_clusters = 12 The average silhouette_score is : 0.3439561998875081
For n_clusters = 13 The average silhouette_score is : 0.347961415981389
For n_clusters = 14 The average silhouette_score is : 0.33782737875946023


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
check_no_sand_mar_preclean=P.profiles.query("location == 'mar' and raw_date==20190205 and label_k in [0,5]")
check_no_sand_leo_preclean=P.profiles.query("location == 'leo' and raw_date==20190211 and label_k ==1")

check_veg_mar_preclean=P.profiles.query("location == 'mar' and raw_date==20180925 and label_k in [1,3]")
check_veg_leo_preclean=P.profiles.query("location == 'leo' and raw_date==20180713 and label_k in [1,5,9]")

check_water_mar_preclean=P.profiles.query("location == 'mar' and raw_date==20190516 and label_k in [4,7]")
check_water_leo_preclean=P.profiles.query("location == 'leo' and raw_date==20180920 and label_k in [0,2,6,7]")

check_sand_mar_preclean=P.profiles.query("location == 'mar' and raw_date==20180621 and label_k in [0,2]")
check_sand_leo_preclean=P.profiles.query("location == 'leo' and raw_date==20190328 and label_k == 3")

check_watermask_mar_pre=P.profiles.query("location == 'mar' and raw_date==20181113 and distance < 10").shape
check_watermask_leo_pre=P.profiles.query("location == 'leo' and raw_date==20180920 and distance < 10").shape

In [9]:
P.cleanit(l_dicts=l_dicts,
          watermasks_path=watermasks_path,
          shoremasks_path=shoremasks_path,
          label_corrections_path=label_corrections_path)

Reclassifying dataset with the provided dictionaries.
Label corrections provided in CRS: epsg:32754
Fine tuning in leo.


C:\anaconda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


  0%|          | 0/6 [00:00<?, ?it/s]

Fine-tuning label_k 3 to no_sand in leo-20180606, found 127 pts.
Fine-tuning label_k 8 to no_sand in leo-20180606, found 73 pts.
Fine-tuning label_k 5 to no_sand in leo-20180713, found 56 pts.
Fine-tuning label_k 6 to no_sand in leo-20180713, found 7 pts.
Fine-tuning label_k 1 to no_sand in leo-20180920, found 89 pts.
Fine-tuning label_k 2 to sand in leo-20180920, found 9 pts.
Fine-tuning label_k 2 to veg in leo-20180920, found 31 pts.
Fine-tuning label_k 6 to sand in leo-20180920, found 17 pts.
Fine-tuning label_k 2 to veg in leo-20190211, found 2 pts.
Fine-tuning label_k 6 to veg in leo-20190211, found 29 pts.
Fine-tuning label_k 6 to no_sand in leo-20190211, found 2 pts.
Fine-tuning label_k 6 to veg in leo-20190211, found 6 pts.
Fine-tuning label_k 6 to sand in leo-20190211, found 28 pts.
Fine-tuning label_k 7 to no_sand in leo-20190211, found 41 pts.
Fine-tuning label_k 0 to sand in leo-20190328, found 6 pts.
Fine-tuning label_k 0 to no_sand in leo-20190328, found 52 pts.
Fine-tuni

  0%|          | 0/6 [00:00<?, ?it/s]

...

Setting to water 643 pts overlapping provided watermasks.
shoremask  provided in CRS: epsg:32754
Applying shoremasks cleaning.
Shoremasking in mar.


C:\anaconda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:
C:\anaconda3\envs\sandpyper_env\lib\site-packages\geopandas\base.py:39: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")
C:\anaconda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:828: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)
C:\anaconda3\envs\sandpyper_env\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prep

Removing 3621 pts falling outside provided shore polygones.
Shoremasking in leo.


C:\anaconda3\envs\sandpyper_env\lib\site-packages\geopandas\base.py:39: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")
C:\anaconda3\envs\sandpyper_env\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Removing 2358 pts falling outside provided shore polygones.
['polygon finetuning', 'watermasking', 'shoremasking'] completed.


in mar, the fourth mask is different

Setting to water 1 pts overlapping provided watermasks. UBUNTU
Setting to water 532 pts overlapping provided watermasks. WINDWOS OK

in mar shoremask mask 14812 in Ubuntu, but should be only 3621 WINDOWS OK

In [10]:
check_watermask_mar_post=P.profiles.query("location == 'mar' and raw_date==20181113 and distance < 10").shape
check_watermask_leo_post=P.profiles.query("location == 'leo' and raw_date==20180920 and distance < 10").shape

check_no_sand_mar_postclean=P.profiles.query("location == 'mar' and raw_date==20190205 and label_k in [0,5]")
check_no_sand_leo_postclean=P.profiles.query("location == 'leo' and raw_date==20190211 and label_k ==1")

check_veg_mar_postclean=P.profiles.query("location == 'mar' and raw_date==20180925 and label_k in [1,3]")
check_veg_leo_postclean=P.profiles.query("location == 'leo' and raw_date==20180713 and label_k in [1,5,9]")

check_water_mar_postclean=P.profiles.query("location == 'mar' and raw_date==20190516 and label_k in [4,7]")
check_water_leo_postclean=P.profiles.query("location == 'leo' and raw_date==20180920 and label_k in [0,2,6,7]")

check_sand_mar_postclean=P.profiles.query("location == 'mar' and raw_date==20180621 and label_k in [0,2]")
check_sand_leo_postclean=P.profiles.query("location == 'leo' and raw_date==20190328 and label_k == 3")

In [13]:
test_slice=P.profiles.query("location=='mar' & survey_date=='2019-05-16'").copy()
test_slice.loc[:,'next_point']=test_slice.loc[:,'coordinates'].shift(1)
test_slice_tr0=test_slice.query(f"tr_id=={test_slice.tr_id.iloc[-1]}").iloc[1:,:].copy()
test_slice_last=test_slice.query(f"tr_id=={max(test_slice.tr_id.unique())}").copy()
test_slice_rand=test_slice.query(f"tr_id=={np.random.randint(min(test_slice.tr_id.unique()),max(test_slice.tr_id.unique()))}").iloc[1:,:].copy()

pts_spacing_0=np.nanmean(test_slice_tr0.coordinates.set_crs(32754).distance(test_slice_tr0.next_point.set_crs(32754)))
pts_spacing_last=np.nanmean(test_slice_last.coordinates.set_crs(32754).distance(test_slice_last.next_point.set_crs(32754)))
pts_spacing_rand=np.nanmean(test_slice_rand.coordinates.set_crs(32754).distance(test_slice_rand.next_point.set_crs(32754)))

In [15]:
(len(opt_k)==P.profiles.groupby(["location","raw_date"]).raw_date.count().shape[0])

True